# Day 2

Today, we will start using nf-core pipelines to find differentially abundant genes in our dataset. 
We are using data from the following paper: https://www.nature.com/articles/s41593-023-01350-3#Sec10

1. Please take some time to read through the paper and understand their approach, hypotheses and goals.

What was the objective of the study?

In [ ]:
#Finding ways to prevent physical dependence and addiction disorders due to misuse of opioid analgesics
#Further they want to transcriptomic 

What do the conditions mean?

oxy: oyxcodone exposure in male mice


sal: control group for oxycodone exposure

What do the genotypes mean?

SNI: spared nerve injury


Sham: no spared nerve injury but with operation (control group for chronic pain)

Imagine you are the bioinformatician in the group who conducted this study. They hand you the raw files and ask you to analyze them.

What would you do?

I would filter and clean the data so only necessary columns and rows are left in the dataset. Next, I would look at any missing data and handle it accordingly. After the data preparation process is finished, I compare different fields of the different groups and look for significant changes according to my hypothesis. E.g. weight loss was mentioned so I would investigate this before, during and after oxycodone exposure.

Which groups would you compare to each other? 

I would compare sham-oxy and sham-sal, sham-oxy and SNI-oxy, and SNI-oxy and SNI-sal

For the example of weight loss, I would expect SNI-oxy and sham-oxy to lose weight during exposure and gain weight afterwards. For the sal groups, I would expect no change.

Please also mention which outcome you would expect to see from each comparison.

Your group gave you a very suboptimal excel sheet (conditions_runs_oxy_project.xlsx) to get the information you need for each run they uploaded to the SRA.<br>
So, instead of directly diving into downloading the data and starting the analysis, you first need to sort the lazy table.<br>
Use Python and Pandas to get the table into a more sensible order.<br>
Then, perform some overview analysis and plot the results
1. How many samples do you have per condition?
2. How many samples do you have per genotype?
3. How often do you have each condition per genotype?

In [4]:
!pwd

/home/lorena/loboehme1/notebooks/day_02


In [9]:
import pandas as pd
sheet = pd.read_excel("conditions_runs_oxy_project.xlsx")
sheet.head()

,Patient,Run,RNA-seq,DNA-seq,condition: Sal,Condition: Oxy,Genotype: SNI,Genotype: Sham
0,?,SRR23195505,x,NaN,x,NaN,x,NaN
1,?,SRR23195506,x,NaN,NaN,x,NaN,x
2,?,SRR23195507,x,NaN,x,NaN,NaN,x
3,?,SRR23195508,x,NaN,NaN,x,x,NaN
4,?,SRR23195509,x,NaN,NaN,x,x,NaN


In [10]:
sheet_dropped = sheet.drop(columns=["Patient", "RNA-seq"])
sheet_dropped.head()

,Run,DNA-seq,condition: Sal,Condition: Oxy,Genotype: SNI,Genotype: Sham
0,SRR23195505,NaN,x,NaN,x,NaN
1,SRR23195506,NaN,NaN,x,NaN,x
2,SRR23195507,NaN,x,NaN,NaN,x
3,SRR23195508,NaN,NaN,x,x,NaN
4,SRR23195509,NaN,NaN,x,x,NaN


In [17]:
sheet_dropped = sheet_dropped.fillna(0)
sheet_dropped = sheet_dropped.replace({"x": 1})
sheet_dropped.head()

,Run,DNA-seq,condition: Sal,Condition: Oxy,Genotype: SNI,Genotype: Sham
1,SRR23195506,0.0,0.0,1.0,0.0,1.0
6,SRR23195511,0.0,0.0,1.0,0.0,1.0
9,SRR23195514,0.0,0.0,1.0,0.0,1.0
14,SRR23195519,0.0,0.0,1.0,0.0,1.0
2,SRR23195507,0.0,1.0,0.0,0.0,1.0


In [18]:
sheet_dropped.sort_values(by=["Genotype: Sham", "Condition: Oxy"], inplace=True)
sheet_dropped

,Run,DNA-seq,condition: Sal,Condition: Oxy,Genotype: SNI,Genotype: Sham
0,SRR23195505,0.0,1.0,0.0,1.0,0.0
5,SRR23195510,0.0,1.0,0.0,1.0,0.0
8,SRR23195513,0.0,1.0,0.0,1.0,0.0
13,SRR23195518,0.0,1.0,0.0,1.0,0.0
3,SRR23195508,0.0,0.0,1.0,1.0,0.0
4,SRR23195509,0.0,0.0,1.0,1.0,0.0
11,SRR23195516,0.0,0.0,1.0,1.0,0.0
12,SRR23195517,0.0,0.0,1.0,1.0,0.0
2,SRR23195507,0.0,1.0,0.0,0.0,1.0
7,SRR23195512,0.0,1.0,0.0,0.0,1.0


In [23]:
count_oxy = sheet_dropped['Condition: Oxy'].sum()
count_sal = sheet_dropped['condition: Sal'].sum()
count_sni = sheet_dropped['Genotype: SNI'].sum()
count_sham = sheet_dropped['Genotype: Sham'].sum()

In [26]:
count_oxy_sni = sheet_dropped[(sheet_dropped['Condition: Oxy'] == 1) & (sheet_dropped['Genotype: SNI'] == 1)].shape[0]
count_oxy_sham = sheet_dropped[(sheet_dropped['Condition: Oxy'] == 1) & (sheet_dropped['Genotype: Sham'] == 1)].shape[0]
count_sal_sni = sheet_dropped[(sheet_dropped['condition: Sal'] == 1) & (sheet_dropped['Genotype: SNI'] == 1)].shape[0]
count_sal_sham = sheet_dropped[(sheet_dropped['condition: Sal'] == 1) & (sheet_dropped['Genotype: Sham'] == 1)].shape[0]

They were so kind to also provide you with the information of the number of bases per run, so that you can know how much space the data will take on your Cluster.<br>
Add a new column to your fancy table with this information (base_counts.csv) and sort your dataframe according to this information and the condition.

Then select the 2 smallest runs from your dataset and download them from SRA (maybe an nf-core pipeline can help here?...)

In [27]:
basecounts = pd.read_csv("base_counts.csv")
basecounts.head()

,Run,Bases
0,SRR23195505,6922564500
1,SRR23195506,7859530800
2,SRR23195507,8063298900
3,SRR23195508,6927786900
4,SRR23195509,7003550100


In [28]:
merged = pd.merge(sheet_dropped, basecounts, left_on="Run", right_on="Run")
merged.head()

,Run,DNA-seq,condition: Sal,Condition: Oxy,Genotype: SNI,Genotype: Sham,Bases
0,SRR23195505,0.0,1.0,0.0,1.0,0.0,6922564500
1,SRR23195510,0.0,1.0,0.0,1.0,0.0,7377388500
2,SRR23195513,0.0,1.0,0.0,1.0,0.0,8099181600
3,SRR23195518,0.0,1.0,0.0,1.0,0.0,7908500400
4,SRR23195508,0.0,0.0,1.0,1.0,0.0,6927786900


In [29]:
merged.sort_values(by=["Bases"], inplace=True)
merged.head()

,Run,DNA-seq,condition: Sal,Condition: Oxy,Genotype: SNI,Genotype: Sham,Bases
6,SRR23195516,0.0,0.0,1.0,1.0,0.0,6203117700
13,SRR23195511,0.0,0.0,1.0,0.0,1.0,6456390900
7,SRR23195517,0.0,0.0,1.0,1.0,0.0,6863840400
0,SRR23195505,0.0,1.0,0.0,1.0,0.0,6922564500
4,SRR23195508,0.0,0.0,1.0,1.0,0.0,6927786900


In [ ]:
#nextflow run nf-core/fetchngs -profile docker --input ids.csv --outdir run_output

While your files are downloading, get back to the paper and explain how you would try to reproduce the analysis.<br>
When you are done with this shout, so we can discuss the different ideas.

I would try to download the data and repeat the steps described in the paper as closely as possible